In [ ]:
import os
import numpy as np
import imageio
import matplotlib
import matplotlib.pyplot as plt

# PATHS TO TRAIN/TEST DATA
data_path = '../data/66_33/'
test_set_path = data_path + 'test/'         # Relative path to test patch root folder
results_set_path = data_path + 'results/'

tiles_total = len(os.listdir(test_set_path + 'CIR/'))

# USE CIR OR RGB DATA
PlotSize = 8                                     # Size of plots
matplotlib.rcParams['figure.figsize'] = [PlotSize*2, PlotSize] 
patch_size = 512

CMAP = matplotlib.colors.ListedColormap(['black', 'white', 'orange'])
CMAP_small = matplotlib.colors.ListedColormap(['black', 'white'])

In [ ]:
array_one = imageio.imread('J:/MP/KRNAP_horni_hranice_lesa/snimky/2018/RGB/ZACLER_5_1.tif').astype(np.uint8)
array_two = imageio.imread('J:/MP/KRNAP_horni_hranice_lesa/snimky/2018/RGB/ZACLER_6_1.tif').astype(np.uint8)
print(array_one.shape)
print(array_two.shape)

In [ ]:
x_11, y_11 = np.histogram(array_one[:,:,0])
x_21, y_21 = np.histogram(array_two[:,:,0])

x_12, y_12 = np.histogram(array_one[:,:,1])
x_22, y_22 = np.histogram(array_two[:,:,1])

x_13, y_13 = np.histogram(array_one[:,:,2])
x_23, y_23 = np.histogram(array_two[:,:,2])

x = np.arange(10)

print(x_11.shape)
print(y_11.shape)

In [ ]:
plt.figure()

plt.subplot(3,2,1)
plt.bar(x, x_11)

plt.subplot(3,2,2)
plt.bar(x, x_21)

plt.subplot(3,2,3)
plt.bar(x, x_12)

plt.subplot(3,2,4)
plt.bar(x, x_22)

plt.subplot(3,2,5)
plt.bar(x, x_13)

plt.subplot(3,2,6)
plt.bar(x, x_23)

In [ ]:
def read_patch(root_folder, cir=True, rgb=True):
    ##########################################################
    # READ IMAGES as FLOAT
    if cir:
        cir_file_list = os.listdir(root_folder + 'CIR/')
        cir_list = []
        
        for file in cir_file_list:
            cir_patch = imageio.imread(root_folder + 'CIR/' + file).astype(np.uint8)
            
            h, w = cir_patch.shape[:2]
            num_samples = h*w
            cir_list.append(cir_patch[:,:,:].reshape((num_samples, 3)))
            del cir_patch

        cir_features = np.concatenate(cir_list, axis=0)
    
    if rgb:
        rgb_file_list = os.listdir(root_folder + 'RGB/')
        rgb_list = []
        
        for file in rgb_file_list:
            rgb_patch = imageio.imread(root_folder + 'RGB/' + file).astype(np.uint8)
            
            h, w = rgb_patch.shape[:2]
            num_samples = h*w
            rgb_list.append(rgb_patch[:,:,:].reshape((num_samples, 3)))
            del rgb_patch
        
        rgb_features = np.concatenate(rgb_list, axis=0)


    if cir and rgb:
        features = np.concatenate([cir_features, rgb_features], axis=1)
    elif cir:
        features = cir_features
    elif rgb:
        features = rgb_features
    else:
        print('No valid data input.')


    gt_file_list = os.listdir(root_folder + 'GT/')
    gt_list = []

    for file in gt_file_list:
        gt_patch = imageio.imread(root_folder + 'GT/' + file).astype(np.uint8)
 
        h, w = gt_patch.shape[:2]
        num_samples = h*w
    
        gt_list.append(gt_patch[:,:].reshape((num_samples)))
        del gt_patch

    ground_truth = np.concatenate(gt_list, axis=0)

    ########################################################## 
    return features, ground_truth

In [ ]:
X, y = read_patch(test_set_path)

In [ ]:
y_mlc = imageio.imread(results_set_path + 'mlc.tif').astype(np.uint8).reshape(tiles_total*patch_size*patch_size)
print('Loaded mlc')

y_svm = imageio.imread(results_set_path + 'svm.tif').astype(np.uint8).reshape(tiles_total*patch_size*patch_size)
print('Loaded svm')

y_rf = imageio.imread(results_set_path + 'rf.tif').astype(np.uint8).reshape(tiles_total*patch_size*patch_size)
print('Loaded rf')

y_unet = imageio.imread(results_set_path + 'U_Net.tif').astype(np.uint8).reshape(tiles_total*patch_size*patch_size)
#y_unet =        np.loadtxt(results_set_path + 'mlc.csv', dtype=np.uint8)
print('Loaded unet')

y_krakonosnet = imageio.imread(results_set_path + 'KrakonosNet.tif').astype(np.uint8).reshape(tiles_total*patch_size*patch_size)
#y_krakonosnet = np.loadtxt(results_set_path + 'mlc.csv', dtype=np.uint8)
print('Loaded krakonosnet')

In [ ]:
def plot(tile_index, data, gt, mlc, svm, rf, unet, krakonosnet, size, num_of_tiles):
    
    # Split the datasets into tiles
    data_list = np.split(data, num_of_tiles)
    gt_list =   np.split(gt,   num_of_tiles)
    mlc_list =  np.split(mlc,  num_of_tiles)
    svm_list =  np.split(svm,  num_of_tiles)
    rf_list =   np.split(rf,   num_of_tiles)
    unet_list = np.split(unet, num_of_tiles)
    krakonos_list =  np.split(krakonosnet, num_of_tiles)
    
    # Select tile by tile_index and reshape to original dimensions
    data = data_list[tile_index].reshape((size,size,data.shape[1]))
    gt   = gt_list[tile_index].reshape((size,size))
    mlc  = mlc_list[tile_index].reshape((size,size))
    svm  = svm_list[tile_index].reshape((size,size))
    rf   = rf_list[tile_index].reshape((size,size))
    unet = unet_list[tile_index].reshape((size,size))
    krakonosnet = krakonos_list[tile_index].reshape((size,size))
    
    
    plt.figure(facecolor='white')
    
    plt.subplot(2, 4, 1)
    plt.imshow(data[:,:,:3])
    plt.title('NIR Red Green composite')
    plt.axis('off')

    plt.subplot(2, 4, 2)
    plt.imshow(data[:,:,-3:])
    plt.title('Red Green Blue composite')
    plt.axis('off')
        
    plt.subplot(2, 4, 3)
    plt.imshow(gt, CMAP)
    plt.title('Annotation')
    plt.axis('off')

    plt.subplot(2, 4, 4)
    plt.imshow(mlc, CMAP)
    plt.title('Maximum Likelihood')
    plt.axis('off')
    
    plt.subplot(2, 4, 5)
    plt.imshow(svm, CMAP)
    plt.title('Support Vector Machine')
    plt.axis('off')
    
    plt.subplot(2, 4, 6)
    plt.imshow(rf, CMAP)
    plt.title('Random Forest')
    plt.axis('off')
    
    plt.subplot(2, 4, 7)
    plt.imshow(unet, CMAP)
    plt.title('U-Net')
    plt.axis('off')
    
    plt.subplot(2, 4, 8)
    plt.imshow(krakonosnet, CMAP)
    plt.title('KrakonosNet')
    plt.axis('off')
    

In [ ]:
plot(110, X, y, y_mlc, y_svm, y_rf, y_unet, y_krakonosnet, patch_size, tiles_total)